In [134]:
# 40
# 係り受け解析結果を読み込む。なお、クラス Morph のリストとして読み込む

import re


class Morph():
    def __init__(self, line):
        if("EOS" in line):
            self.surface = "\n"
            self.base = "\n"
            self.pos = "EOS"
            self.pos1 = "EOS"
        elif(line[0] == "*"):
            pass
        else:
            tmp = re.split("[\t\n,]", line)
            self.surface = tmp[0]
            self.base = tmp[7]
            self.pos = tmp[1]
            self.pos1 = tmp[2]

with open("ai.ja.txt.parsed", "r", encoding="utf-8") as f:
    data = f.readlines()

morfemes = [Morph(line) for line in data if(not line[0] == "*")]
for i in range(15):
    print(morfemes[i].surface, end=" ")



人工 知能 
 
 人工 知能 （ じん こうち のう 、 、 AI 〈 エーアイ 

In [146]:
# 41
# 係り受け解析結果を読み込んで、分節を著すクラス Chunk を作成する。
# 形態素のリスト morphs、係り先文節インデックス番号 dst、係り元分節インデックス番号 srcs を持つ

import re


class Chunk():
    def __init__(self, data):
        self.dst = int(re.sub("D", "", re.split("\s", data[0])[2]))
        self.morphs = [Morph(line) for line in data[1:]]
        self.srcs = []
        # srcs はクラス Sentence で定義する

        self.phrase = ''
        self.noun = False
        for morph in self.morphs:
            if morph.pos != '記号':
                self.phrase += morph.surface # 記号以外の場合文節作成
            if morph.pos == '名詞':
                self.noun = True


class Sentence():
    def __init__(self, data):
        self.chunks = []
        chunk = []
        for line in data:
            if((line[0] == "*" or "EOS" in line) and chunk != []):
                self.chunks.append(Chunk(chunk))
                chunk = list()
            chunk.append(line)
        for i in range(len(self.chunks)):
            dst = self.chunks[i].dst
            self.chunks[dst].srcs.append(i)
            

def make_sentences(data):
    sentences = []
    sentence = []
    for line in data:
        if(("EOS" in line) and (sentence != [])):
            sentence.append(line)
            sentences.append(Sentence(sentence))
            sentence = list()
        elif(not "EOS" in line):
            sentence.append(line)
    return sentences

with open("ai.ja.txt.parsed", "r", encoding="utf-8") as f:
    data = f.readlines()

sentences = make_sentences(data)

for i in range(len(sentences)):
    for j in range(len(sentences[i].chunks)):
        for k in range(len(sentences[i].chunks[j].morphs)):
            print(sentences[i].chunks[j].morphs[k].surface)
        print("----------係り先:", sentences[i].chunks[j].dst)

人工
知能
----------係り先: -1
人工
知能
----------係り先: 17
（
じん
こうち
のう
、
、
----------係り先: 17
AI
----------係り先: 3
〈
エーアイ
〉
）
と
は
、
----------係り先: 17
「
『
計算
----------係り先: 5
（
）
』
という
----------係り先: 9
概念
と
----------係り先: 9
『
コンピュータ
----------係り先: 8
（
）
』
という
----------係り先: 9
道具
を
----------係り先: 10
用い
て
----------係り先: 12
『
知能
』
を
----------係り先: 12
研究
する
----------係り先: 13
計算
機
科学
----------係り先: 14
（
）
の
----------係り先: 15
一
分野
」
を
----------係り先: 16
指す
----------係り先: 17
語
。
----------係り先: 34
「
言語
の
----------係り先: 20
理解
や
----------係り先: 20
推論
、
----------係り先: 21
問題
解決
など
の
----------係り先: 22
知的
行動
を
----------係り先: 24
人間
に
----------係り先: 24
代わっ
て
----------係り先: 26
コンピューター
に
----------係り先: 26
行わ
せる
----------係り先: 27
技術
」
、
または
、
----------係り先: 34
「
計算
機
----------係り先: 29
（
コンピュータ
）
による
----------係り先: 31
知的
な
----------係り先: 31
情報処理
システム
の
----------係り先: 33
設計
や
----------係り先: 33
実現
に関する
----------係り先: 34
研究
分野
」
と
も
----------係り先: 35
さ
れる
。
----------係り先: -1
『
日本
大
百科全書
(
ニッポニカ
)』
の
----------係り先: 1
解説
で
、


In [136]:
# 42 
# 係り元と係り先の分節を表示する

for i in range(len(sentences)):
    for j in range(len(sentences[i].chunks)):
        for k in range(len(sentences[i].chunks[j].morphs)):
            print(sentences[i].chunks[j].morphs[k].surface, end="")
        dst = sentences[i].chunks[j].dst
        print("\t", end="")
        if(dst != -1):
            for l in range(len(sentences[i].chunks[dst].morphs)):
                print(sentences[i].chunks[dst].morphs[l].surface, end="")
        print("\n")

人工知能	

人工知能	語。

（じんこうちのう、、	語。

AI	〈エーアイ〉）とは、

〈エーアイ〉）とは、	語。

「『計算	（）』という

（）』という	道具を

概念と	道具を

『コンピュータ	（）』という

（）』という	道具を

道具を	用いて

用いて	研究する

『知能』を	研究する

研究する	計算機科学

計算機科学	（）の

（）の	一分野」を

一分野」を	指す

指す	語。

語。	研究分野」とも

「言語の	推論、

理解や	推論、

推論、	問題解決などの

問題解決などの	知的行動を

知的行動を	代わって

人間に	代わって

代わって	行わせる

コンピューターに	行わせる

行わせる	技術」、または、

技術」、または、	研究分野」とも

「計算機	（コンピュータ）による

（コンピュータ）による	情報処理システムの

知的な	情報処理システムの

情報処理システムの	実現に関する

設計や	実現に関する

実現に関する	研究分野」とも

研究分野」とも	される。

される。	

『日本大百科全書(ニッポニカ)』の	解説で、

解説で、	述べている。

情報工学者・通信工学者の	佐藤理史は

佐藤理史は	述べている。

次のように	述べている。

述べている。	

人間の	知的能力を

知的能力を	実現する、

コンピュータ上で	実現する、

実現する、	技術・ソフトウェア・コンピュータシステム。

様々な	技術・ソフトウェア・コンピュータシステム。

技術・ソフトウェア・コンピュータシステム。	ある。

応用例は	ある。

自然言語処理	（機械翻訳・かな漢字変換・構文解析等）、

（機械翻訳・かな漢字変換・構文解析等）、	専門家の

専門家の	推論・判断を

推論・判断を	模倣する

模倣する	エキスパートシステム、

エキスパートシステム、	画像認識等が

画像データを	解析して

解析して	検出・抽出したりする

特定の	パターンを

パターンを	検出・抽出したりする

検出・抽出したりする	画像認識等が

画像認識等が	ある。

ある。	命名された。

1956年に	命名された。

ダートマス会議で	命名された。

ジョン・マッカーシーにより	命名された。

命名された。	使われている。

現在では、	使われている。

In [137]:
# 43
# 名詞を含む文節が動詞を含む文節に係る者を抽出


def is_noun(morphs):
    is_noun_list = [(m.pos == "名詞") for m in morphs]
    return (True in is_noun_list)

def is_verb(morphs):
    is_verb_list = [(m.pos == "動詞") for m in morphs]
    return (True in is_verb_list)


for i in range(len(sentences)):
    for j in range(len(sentences[i].chunks)):
        dst = sentences[i].chunks[j].dst

        # if文: 名詞を含む文節か？ and 動詞を含む文節か？
        if(is_noun(sentences[i].chunks[j].morphs) and is_verb(sentences[i].chunks[dst].morphs)):

            for k in range(len(sentences[i].chunks[j].morphs)):
                print(sentences[i].chunks[j].morphs[k].surface, end="")
            print("\t", end="")
            if(dst != -1):
                for l in range(len(sentences[i].chunks[dst].morphs)):
                    print(sentences[i].chunks[dst].morphs[l].surface, end="")
            print("\n")


道具を	用いて

『知能』を	研究する

一分野」を	指す

知的行動を	代わって

人間に	代わって

コンピューターに	行わせる

研究分野」とも	される。

解説で、	述べている。

佐藤理史は	述べている。

次のように	述べている。

知的能力を	実現する、

コンピュータ上で	実現する、

技術・ソフトウェア・コンピュータシステム。	ある。

応用例は	ある。

推論・判断を	模倣する

画像データを	解析して

解析して	検出・抽出したりする

パターンを	検出・抽出したりする

画像認識等が	ある。

1956年に	命名された。

ダートマス会議で	命名された。

ジョン・マッカーシーにより	命名された。

命名された。	使われている。

現在では、	使われている。

記号処理を	用いた

記述を	する

主体と	する

意味あいでも	使われている。

思考ルーチンも	呼ばれる

ことも	ある。

カウンセラーを	模倣した

プログラム	出されるが、

（人工無脳）が	出されるが、

引き合いに	出されるが、

計算機に	させようという

役割を	させようという

「エキスパートシステム」と	呼ばれる

実現は、	困難視されている。

人間が	持つ

暗黙に	持つ

記述が	なり、

問題と	なり、

利用が	困難視されている。

困難視されている。	ある。

アプローチとしては、	知られているが、

アプローチも	知られているが、

差は	ある。

記号的明示性に	ある。

その後	集めた。

「サポートベクターマシン」が	集めた。

注目を	集めた。

経験を	行う

元に	行う

学習を	行う

手法も	ある。

宇宙において、	実装するという

言葉通り、	実装するという

知性を	表現し

機械的に	表現し

表現し	実装するという

登場により、	行った。

流行を	超えて

社会に	浸透して

浸透して	行った。

2017年にかけて、	導入した

ディープラーニングを	導入した

プレイヤーも	破り、

麻雀では	なった。

「MicrosoftSuphx(SuperPhoenix)」が	到達するなど、

AIとして	到達するなど、

十段に	到達するなど、

到達するなど、	なった。

最先端技術と	なった。

人工知能は	呼ばれ、

In [138]:
# 44
# 係り受け木の可視化

import graphviz


kakariuke_graph = graphviz.Digraph()

# エッジの生成
for i in range(len(sentences)):
    for j in range(len(sentences[i].chunks)):
        dst = sentences[i].chunks[j].dst
        a = ""
        b = ""
        for k in range(len(sentences[i].chunks[j].morphs)):
            a += sentences[i].chunks[j].morphs[k].surface
        for k in range(len(sentences[i].chunks[dst].morphs)):
            b += sentences[i].chunks[dst].morphs[k].surface
        
        kakariuke_graph.edge(a, b)


kakariuke_graph.format = "png"
kakariuke_graph.attr("edge", fontname="MS Gothic", encoding='utf-8')
kakariuke_graph.attr("node", fontname="MS Gothic", encoding='utf-8')
kakariuke_graph.render(view=True)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.514622 to fit


'Digraph.gv.png'

In [139]:
# 45
# 述語とそれに対する格の抽出

verbs_cases = {}


def is_noun(morphs):
    is_noun_list = [(m.pos == "名詞") for m in morphs]
    return (True in is_noun_list)

def is_verb(morphs):
    is_verb_list = [(m.pos == "動詞") for m in morphs]
    return (True in is_verb_list)


for i in range(len(sentences)):
    for j in range(len(sentences[i].chunks)):
        dst = sentences[i].chunks[j].dst

        # if文: 名詞を含む文節か？ and 動詞を含む文節か？
        if(is_noun(sentences[i].chunks[j].morphs) and is_verb(sentences[i].chunks[dst].morphs)):
            if(dst != -1):
                case = sentences[i].chunks[j].morphs[-1]
                verb = sentences[i].chunks[dst].morphs[0]
                
                case_normalized = case.base
                verb_normalized = verb.base


                # サ変名詞かどうかの判定
                if(verb.pos == "名詞"):
                    verb_normalized += sentences[i].chunks[dst].morphs[1].base
                
                if(case.pos == "助詞"):
                    if(verb_normalized in verbs_cases):
                        # if(not case_normalized in verbs_cases[verb_normalized]):
                            verbs_cases[verb_normalized].append(case_normalized)
                    else:
                        verbs_cases[verb_normalized] = [case_normalized]

for k in verbs_cases.keys():
    verbs_cases[k].sort()

verbs_cases_str = ""
for k, vs in verbs_cases.items():
    verbs_cases_str += k + "\t"
    for v in vs:
        verbs_cases_str += v + " "
    verbs_cases_str += "\n"

with open("verbs_cases.txt", "w") as f:
    f.write(verbs_cases_str)

In [140]:
# 46
# 述語、格、項を抽出する

verbs_cases_args = {}


def is_noun(morphs):
    is_noun_list = [(m.pos == "名詞") for m in morphs]
    return (True in is_noun_list)

def is_verb(morphs):
    is_verb_list = [(m.pos == "動詞") for m in morphs]
    return (True in is_verb_list)


for i in range(len(sentences)):
    for j in range(len(sentences[i].chunks)):
        dst = sentences[i].chunks[j].dst

        # if文: 名詞を含む文節か？ and 動詞を含む文節か？
        if(is_noun(sentences[i].chunks[j].morphs) and is_verb(sentences[i].chunks[dst].morphs)):
            if(dst != -1):
                verb = sentences[i].chunks[dst].morphs[0]
                case = sentences[i].chunks[j].morphs[-1]
                arg = "".join([sentences[i].chunks[j].morphs[k].surface for k in range(len(sentences[i].chunks[j].morphs)) ])

                case_normalized = case.base
                verb_normalized = verb.base


                # サ変名詞かどうかの判定
                if(verb.pos == "名詞"):
                    verb_normalized += sentences[i].chunks[dst].morphs[1].base
                
                if(case.pos == "助詞"):
                    if(verb_normalized in verbs_cases_args):
                        verbs_cases_args[verb_normalized][0].append(case_normalized)
                        verbs_cases_args[verb_normalized][1].append(arg)
    
                    else:
                        verbs_cases_args[verb_normalized] = []
                        verbs_cases_args[verb_normalized].append([case_normalized])
                        verbs_cases_args[verb_normalized].append([arg])


verbs_cases_args_str = ""

for k, vs in verbs_cases_args.items():
    verbs_cases_args_str += k + "\t"
    for case in vs[0]:
        verbs_cases_args_str += case + " "
    verbs_cases_args_str += "\t"
    for arg in vs[1]:
        verbs_cases_args_str += arg + " "
    verbs_cases_args_str += "\n"

with open("verbs_cases_args.txt", "w", encoding="utf-8") as f:
    f.write(verbs_cases_args_str)


In [141]:
# 47
# Q46 に関して、サ変接続名詞 + 「を」 + 動詞 という形になる文字列についてのみ行う


verbs_cases_args = {}


def is_noun(morphs):
    is_noun_list = [(m.pos == "名詞") for m in morphs]
    return (True in is_noun_list)

def is_verb(morphs):
    is_verb_list = [(m.pos == "動詞") for m in morphs]
    return (True in is_verb_list)


for i in range(len(sentences)):
    for j in range(len(sentences[i].chunks)):
        dst = sentences[i].chunks[j].dst

        # if文: 名詞を含む文節か？ and 動詞を含む文節か？
        if(is_noun(sentences[i].chunks[j].morphs) and is_verb(sentences[i].chunks[dst].morphs)):
            if(dst != -1):
                verb = sentences[i].chunks[dst].morphs[0]
                case = sentences[i].chunks[j].morphs[-1]
                arg = "".join([sentences[i].chunks[j].morphs[k].surface for k in range(len(sentences[i].chunks[j].morphs)) ])


                case_normalized = case.base
                verb_normalized = verb.base

                # 文節がサ変名詞かどうかの判定
                sahen_arg = []
                for k in range(len(sentences[i].chunks[j].morphs)):
                    if(sentences[i].chunks[j].morphs[k].surface == "を" 
                    and sentences[i].chunks[j].morphs[k-1].pos1 == "サ変接続"):
                        for l in range(len(sentences[i].chunks[j].morphs)):
                            sahen_arg.append(sentences[i].chunks[j].morphs[l].base)
                sahen_arg = "".join(sahen_arg)

                if(len(sahen_arg) >= 1):
                    # サ変名詞かどうかの判定 (サ変名詞 + する の形に変換する)
                    if(verb.pos == "名詞"):
                        verb_normalized += sentences[i].chunks[dst].morphs[1].base
                    
                    verb_normalized = sahen_arg + verb_normalized

                    if(case.pos == "助詞"):
                        if(verb_normalized in verbs_cases_args):
                            verbs_cases_args[verb_normalized][0].append(case_normalized)
                            verbs_cases_args[verb_normalized][1].append(arg)
                        else:
                            verbs_cases_args[verb_normalized] = []
                            verbs_cases_args[verb_normalized].append([case_normalized])
                            verbs_cases_args[verb_normalized].append([arg])

print(verbs_cases_args)

{'知的行動を代わる': [['を'], ['知的行動を']], '推論・判断を模倣する': [['を'], ['推論・判断を']], '記号処理を用いる': [['を'], ['記号処理を']], '記述をする': [['を'], ['記述を']], '注目を集める': [['を', 'を', 'を'], ['注目を', '注目を', '注目を']], '経験を行う': [['を'], ['経験を']], '学習を行う': [['を'], ['学習を']], '流行を超える': [['を'], ['流行を']], '学習を繰り返す': [['を'], ['学習を']], '統計的学習を生成する': [['を'], ['統計的学習を']], '進化を見せる': [['を'], ['進化を']], 'コンテンツ生成を行う': [['を'], ['コンテンツ生成を']], '開発を行う': [['を'], ['開発を']], 'テストを可能に': [['を'], ['テストを']], '処理を行う': [['を'], ['処理を']], '知的処理を行う': [['を', 'を'], ['知的処理を', '知的処理を']], '意味を付加する': [['を'], ['意味を']], '研究を進める': [['を'], ['研究を']], '命令を解釈する': [['を'], ['命令を']], '運転を最適化': [['を', 'を'], ['運転を', '運転を']], '特許を取得する': [['を'], ['特許を']], '研究を開始する': [['を'], ['研究を']], '*制御を開発する': [['を'], ['ニューロファジィ制御を']], '成功を受ける': [['を'], ['成功を']], '知的制御を用いる': [['を'], ['知的制御を']], '制御を改善する': [['を'], ['制御を']], '知的制御を搭載する': [['を'], ['知的制御を']], '進歩を担う': [['を'], ['進歩を']], '精度改善を果たす': [['を'], ['精度改善を']], '専用プログラムを使う': [['を'], ['専用プログラムを']], '研究を続ける': [['を'], ['研究を']], '記号接地問題**問題*を

In [148]:
# 48
# 名詞から構文木の根へ至るパスを抽出する



for i, sentence in enumerate(sentences):
    for chunk in sentence.chunks:
        if chunk.noun and chunk.dst != -1:
            line = chunk.phrase
            current_chunk = chunk
            while(current_chunk.dst != -1):
                line += " -> " + sentence.chunks[current_chunk.dst].phrase
                current_chunk = sentence.chunks[current_chunk.dst]
            print(i, '\t',line)
    if(i > 10):
        break


1 	 人工知能 -> 語 -> 研究分野とも -> される
1 	 じんこうちのう -> 語 -> 研究分野とも -> される
1 	 AI -> エーアイとは -> 語 -> 研究分野とも -> される
1 	 エーアイとは -> 語 -> 研究分野とも -> される
1 	 計算 -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 概念と -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 コンピュータ -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 知能を -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 一分野を -> 指す -> 語 -> 研究分野とも -> される
1 	 語 -> 研究分野とも -> される
1 	 言語の -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
1 	 理解や -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
1 	 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
1 	 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
1 	 知的行動を -> 代